# 协程和异步I/O

## 爬虫中可能会遇到障碍
- 盗链接
    - 不通过别人页面直接访问别人的数据
- 防盗链接
- 中间件 - 'www.jd.com' in request.META['HTTP_Referer'],以django为例
- 判断这个域名是否在请求头中，如果在说明不是盗链接

- 反防盗链接
- 强行在自己的请求头时，在自己请求头里加上在访问这个页面上一个页面的url
- requests.get(url, headers={'HTTP_Referer': 'https://ww.abc.com/video'}



- 如果在页面的源码里有这段代码表示页面再进来时会跳转到后面的链接中，需要取到后面的链接再进行访问

```html
<head>
    <meta http-equiv="refresh" content="5;https://www.baidu.com">
</head>

```

- 如果遇到像京东页面需要向下滚动才会加载页面的页面时可以用下面的方法

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from time import sleep


def main():
    driver = webdriver.Chrome()
    driver.get('https://www.jd.com')
    # 里面可以执行js代码，通过js代码来控制浏览器
    driver.execute_script('document.documentElement.scrollTop = 10000')
    while True:
        sleep(1)



if __name__ == '__main__':
    main()

## 并发编程
- 多线程 + GIL（全局解释器锁，无法发挥cpu的多核特性）+ 共享内存 适合：I/O密集型
- 多进程 + IPC（内部进程通信机制， socket/ pipe）适合：数据密集型
- 多进程 + 多线程 适合：数据密集型
- 多进程 + 微线程（协程）适合：数据密集型
- 单线程 + 异步I/O 适合：I/O密集型
- node.js

- 在linux或mac下
- os.fork()可以创建子进程，并返回进程号
    - 主进程返回子进程的进程号，子进程返回0

### 多进程
- 使用多进程对I/O密集型任务的性能没有实质性的改善
- 多进程可以利用cpu的多核特性对计算密集型任务有用
- 对于网络爬虫这种I/O密集型任务还有一种非常好的方式就是单线程+异步I/O
- 所谓的异步I/O就是非阻塞式的I/O操作，也就是在进行I/O操作时没有终段CPU

### 多线程
- 多线程编程最担心什么？ - 多个线程竞争资源
- 如果要保证数据的安全性就需要加锁进行保护threading.Lock
- acquire() / release()
- 如果多个线程获取锁的方式不正确那么就有可能导致死锁（dead lock）
- 加锁就意味着线程需要排队等待， 相当于并发代码变成了串行执行
- 所以在写多线程代码可以尽量避开对资源的竞争
- 一种做法就是通过ThreadLocal为线程绑定资源让每个线程持有资源的副本

### 协程
- 两个函数在互不调用的情况下相互协作
- 在一个线程里，没有切换，不会有资源的竞争
- 协程是一个消费型的生成器


In [ ]:
# 基本的协程
from time import sleep
# 查看协程的状态
from inspect import getgeneratorstate


def countdown_gen(n, consumer):
    # 预激 -- 预先激活
    # GEN_CREATED
    print(getgeneratorstate(consumer))
    consumer.send(None)
    # 所谓的激活就是让代码执行到yield表达式
    # GEN_SUSPEND
    print(getgeneratorstate(consumer))
    while n > 0:
        # 将n发送给这个消费者
        consumer.send(n)
        n -= 1
    try:
        consumer.send(None)
    except StopIteration:
        pass
    finally:
        # 释放
        consumer.close()
        getgeneratorstate(consumer)

def countdown_con():
    while True:
        # 将上面发送过来的数，赋给n
        n = yield
        if n:
            print('Countdown:', n)
            sleep(1)
        else:
            break



def main():
    consumer = countdown_con()
    countdown_gen(10, consumer)


if __name__ == '__main__':
    main()


In [ ]:
from time import sleep
from myutils import coroutine


@coroutine
def create_delivery_man(name, capacity=1):
    buffer = []
    while True:
        size = 0
        while size < capacity:
            pkg_name = yield
            if pkg_name:
                size += 1
                buffer.append(pkg_name)
                print('%s正在接收%s' % (name, pkg_name))
            else:
                break
        print('%s正在派送%d件包裹' % (name, len(buffer)))
        sleep(3)
        buffer.clear()


def create_package_center(consumer, max_packages):
    # consumer.send(None)
    num = 1
    while num <= max_packages:
        print('快递中心准备派送%d号包裹' % num)
        consumer.send('包裹-%d' % num)
        num += 1
        if num % 10 == 0:
            print('接货中。。。。')
            sleep(5)


def main():
    dm = create_delivery_man('王大锤', 7)
    create_package_center(dm, 50)


if __name__ == '__main__':
    main()

In [ ]:
"""
看 yield from 语法
scapy 
movie.douban,com
豆瓣电影

"""